In [1]:
import os
import pandas as pd
from collections import Counter

# 1. Check the number of label files to be processed

In [2]:
path = './IEMOCAP_Label/'
files = os.listdir(path)
print(len(files))

1


# 2. Filter the four information : required emotion label, start and end time, file name, emotion dimension 

In [3]:
# These emotion will be select.
emotion = ['hap', 'exc', 'ang', 'sad', 'neu'] 

sel_emo = []
sel_name = []
sel_dim = []
sel_time = []
sel_start = []
all_emo = []

for file in files:
    label_path = os.path.join(path, file)
    with open(label_path, 'r') as rf:
        lines = rf.readlines()
        data = [[a for a in cur_data.split('\t')] for cur_data in lines]
        for i in range(len(data)-1):
            if data[i][0]=='\n':
                n = i+1
                all_emo.append(data[n][2])
                if data[n][2] in emotion:
                    sel_time.append(data[n][0])
                    sel_start.append(data[n][0].split('-')[0][1:6])
                    sel_name.append(data[n][1])
                    sel_emo.append(data[n][2])
                    sel_dim.append(data[n][3].split('\n')[0])

print('The number of all emotion in lable file: {}'.format(len(all_emo)))
print(Counter(all_emo), '\n')

print('The number of selected emotion in lable file: {}'.format(len(sel_emo)))
print(Counter(sel_emo))

The number of all emotion in lable file: 30
Counter({'fru': 18, 'neu': 5, 'xxx': 4, 'ang': 3}) 

The number of selected emotion in lable file: 8
Counter({'neu': 5, 'ang': 3})


# 3. Create label file

In [5]:
df = pd.DataFrame(columns=('time', 'data_name', 'emotion_categorical', 'emotion_dimension'))
df['time'] = sel_time
df['data_name'] = sel_name
df['emotion_categorical'] = sel_emo
df['emotion_dimension'] = sel_dim
df['start'] = sel_start
df['start'] = df['start'].apply(pd.to_numeric)

# Merging the happiness and excitement categories as the happy category.
df.loc[df['emotion_categorical']=='exc', 'emotion_categorical'] = 'hap'

# First sort the Session, and then sort the speaking time.
files = ['Ses01F_impro01.txt']
files.sort()
df['session'] = [a[:-5] for a in df['data_name']]
df_new = pd.DataFrame(columns=('time', 'data_name', 'emotion_categorical', 'emotion_dimension', 'start', 'session'))
for i in files:
    block = df.loc[df['session']==i[:-4]]
    block = block.sort_values(['start'], ascending=True)
    df_new = df_new.append(block)

# creat .csv file    
print('Finally, select the number of data is: {}'.format(df_new.shape[0]))
df_new.to_csv('./IEMOCAP4.csv', index=False)

Finally, select the number of data is: 8
